In [1]:
import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/Research/PaperCode/stylised_facts/')

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import lob_for_futures as lobFut
from lob_for_futures import *
import os
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
import pingouin as pig
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import matplotlib.ticker as mtick
dataFolder = lobFut.dataFolder
quotes = [f for f in os.listdir(dataFolder) if str('_quotes') in f]
trades = [f for f in os.listdir(dataFolder) if str('_trades') in f]
elements = '/media/ak/Elements/OrderBookProcessedData'

procesSymbols = sorted(os.listdir(elements))
symbolIdx = 0

symbols = [f.split("_")[0] for f in quotes]
#
symbol = sorted(symbols)[symbolIdx]
print(symbol)
quotesFileCh = os.path.join(dataFolder, quotes[symbolIdx])
tradesFileCh = os.path.join(dataFolder, trades[symbolIdx])
#
# # get common Dates
quotesDates = sorted([f.split(".csv")[0] for f in os.listdir(quotesFileCh)])
tradesDates = sorted([f.split(".csv")[0] for f in os.listdir(tradesFileCh)])
intersectionDates = list(set(quotesDates).intersection(tradesDates))

trades_cols = ['size', 'time', 'type', 'value']
#experimentsDestination = '/media/ak/T7/MFDFA Experiments'
plt.style.use(os.path.join('/home/ak/.config/matplotlib','latexstyle3.mplstyle'))

DU1


In [12]:
dataFolder

'/media/ak/WorkDrive/Data'

In [13]:
def returns(s):

    # compute log returns
    arr = np.diff(np.log(s))

    return (pd.Series(arr, index=s.index[1:]))

In [14]:

symbols = sorted(os.listdir(elements))
procesSymbols

['DU1', 'JB1', 'KE1', 'OE1', 'RX1', 'TU1', 'US1']

In [15]:
# Lets create a function that shifted your dataframe first before calling the corr(). 
# Let us break down what we hope to accomplish, and then translate that into code. 
# For each hour (row) and for a given feature (column) I would like to find the value 
# for that feature N hours prior. 
# For each value of N (1-6 in our case) I want to make a new column 
# for that feature representing the Nth prior hour's measurement.

def df_derived_by_shift(df,lag=0,NON_DER=[]):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in NON_DER:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pd.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=i)
            i+=1
        df = pd.concat([df, dfn], axis=1)
    return df

def get_test_stats(bar_types,bar_returns,test_func,*args,**kwds):
    
    dct = {bar:(int(bar_ret.shape[0]), test_func(bar_ret,*args,**kwds)) 
           for bar,bar_ret in zip(bar_types,bar_returns)}
    df = (pd.DataFrame.from_dict(dct)
          .rename(index={0:'sample_size',1:f'{test_func.__name__}_stat'})
          .T)
    return df

In [16]:
symbolIdx = 1
procsdSymbolFolder = os.path.join(elements, procesSymbols[symbolIdx])
files = sorted(os.listdir(procsdSymbolFolder))
bars =['volume_bar', 'calendar_bar', 'usd_volume_bar', 'tick_bar']
symbol = symbols[symbolIdx]
print(symbol)

JB1


In [17]:
filesIdx =0
fileLocation = os.path.join(procsdSymbolFolder, files[filesIdx])
openedDict = open_pickle_filepath(fileLocation)['volume_bar']

In [18]:
volume_bar_df = openedDict[list(openedDict.keys())[0]]

In [19]:
apply_micro_structure_features(volume_bar_df)

Applying micro-structure features


,volume_bucket,BidSize_open,BidSize_high,BidSize_low,BidSize_close,QuoteTime_open,QuoteTime_high,QuoteTime_low,QuoteTime_close,type_x_open,type_x_high,type_x_low,type_x_close,BestBid_open,BestBid_high,BestBid_low,BestBid_close,TradeTime_open,TradeTime_high,TradeTime_low,TradeTime_close,AskSize_open,AskSize_high,AskSize_low,AskSize_close,time_y_open,time_y_high,time_y_low,time_y_close,type_y_open,type_y_high,type_y_low,type_y_close,BestAsk_open,BestAsk_high,BestAsk_low,BestAsk_close,TradeSize_open,TradeSize_high,TradeSize_low,TradeSize_close,type_open,type_high,type_low,type_close,TradePrice_open,TradePrice_high,TradePrice_low,TradePrice_close,TimeStamp_open,TimeStamp_high,TimeStamp_low,TimeStamp_close,TradeVolume_open,TradeVolume_high,TradeVolume_low,TradeVolume_close,total_traded_volume_open,total_traded_volume_high,total_traded_volume_low,total_traded_volume_close,milliSeconds_open,milliSeconds_high,milliSeconds_low,milliSeconds_close,dollar_traded_volume_open,dollar_traded_volume_high,dollar_traded_volume_low,dollar_traded_volume_close,timeStampIdx_open,timeStampIdx_high,timeStampIdx_low,timeStampIdx_close,micro_price_open,micro_price_high,micro_price_low,micro_price_close,weighted_average_BestBid,weighted_average_BestAsk,weighted_activity_spread,total_size,micro_price,price_imbalance,pct_change_micro_price,simple_mid_price
0,1.0,114,114,114,114,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,BID,BID,BID,BID,150.88,150.88,150.88,150.88,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,9,9,9,9,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,ASK,ASK,ASK,ASK,150.89,150.89,150.89,150.89,133.0,133.0,133.0,133.0,TRADE,TRADE,TRADE,TRADE,150.89,150.89,150.89,150.89,06:30:01,06:30:01,06:30:01,06:30:01,133.0,133.0,133.0,133.0,133.0,133.0,133.0,133.0,23401000,23401000,23401000,23401000,20068.37,20068.37,20068.37,20068.37,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,2018-04-13 06:30:01,150.880732,150.880732,150.880732,150.880732,150.880000,150.890000,0.010000,492,150.880732,-128.799268,NaN,150.885000
1,2.0,110,129,1,116,2018-04-13 06:30:01,2018-04-13 06:30:18,2018-04-13 06:30:01,2018-04-13 06:30:18,BID,BID,BID,BID,150.88,150.89,150.88,150.88,2018-04-13 06:30:01,2018-04-13 06:30:18,2018-04-13 06:30:01,2018-04-13 06:30:18,9,134,5,35,2018-04-13 06:30:01,2018-04-13 06:30:18,2018-04-13 06:30:01,2018-04-13 06:30:18,ASK,ASK,ASK,ASK,150.89,150.90,150.89,150.89,133.0,133.0,1.0,2.0,TRADE,TRADE,TRADE,TRADE,150.89,150.89,150.89,150.89,06:30:01,06:30:18,06:30:01,06:30:18,133.0,133.0,1.0,2.0,133.0,133.0,1.0,2.0,23401000,23418000,23401000,23418000,20068.37,20068.37,150.89,301.78,2018-04-13 06:30:01,2018-04-13 06:30:18,2018-04-13 06:30:01,2018-04-13 06:30:18,150.880756,150.899919,150.880373,150.882318,150.883624,150.897322,0.013699,539,150.888275,-48.423673,1.051276e-05,150.888095
2,3.0,115,146,112,114,2018-04-13 06:30:19,2018-04-13 06:32:32,2018-04-13 06:30:19,2018-04-13 06:32:32,BID,BID,BID,BID,150.88,150.88,150.88,150.88,2018-04-13 06:30:19,2018-04-13 06:32:32,2018-04-13 06:30:19,2018-04-13 06:32:32,36,93,36,90,2018-04-13 06:30:19,2018-04-13 06:32:32,2018-04-13 06:30:19,2018-04-13 06:32:32,ASK,ASK,ASK,ASK,150.89,150.89,150.89,150.89,2.0,10.0,1.0,10.0,TRADE,TRADE,TRADE,TRADE,150.89,150.89,150.88,150.88,06:30:19,06:32:32,06:30:19,06:32:32,2.0,10.0,1.0,10.0,2.0,10.0,1.0,10.0,23419000,23552000,23419000,23552000,301.78,1508.80,150.88,1508.80,2018-04-13 06:30:19,2018-04-13 06:32:32,2018-04-13 06:30:19,2018-04-13 06:32:32,150.882384,150.884439,150.882384,150.884412,150.880000,150.890000,0.010000,742,150.883437,-47.171981,1.387760e-05,150.885000
3,4.0,116,128,112,116,2018-04-13 06:32:34,2018-04-13 06:35:00,2018-04-13 06:32:34,2018-04-13 06:35:00,BID,BID,BID,BID,150.88,150.88,150.88,150.88,2018-04-13 06:32:34,2018-04-13 06:35:00,2018-04-13 06:32:34,2018-04-13 06:35:00,90,94,88,94,2018-04-13 06:32:34,2018-04-13 06:35:00,2018-04-13 06:32:34,2018-04-13 

In [20]:
!pip install lead-lag

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ak/anaconda3/envs/mmd-kernels/lib/python3.7/site-packages/matplotlib-3.4.3.dist-info/METADATA'

